# M7_AE3_P2: Ejecuta un Job Spark completo con acciones y monitoreo

## Contexto
Conocer el ciclo de ejecución de un Job Spark (transformaciones → acción → ejecución) permite entender el flujo completo de procesamiento distribuido.

## Consigna
Simula la ejecución de un job Spark desde la carga de datos hasta la ejecución de una acción como `mean()` o `sum()`.  
Identifica cuándo se desencadena realmente el procesamiento y cómo se vería el Job internamente (DAG, Stages y Tasks).

## Paso a Paso

1. **Carga un RDD numérico** desde una lista o un archivo.
2. **Aplica una o más transformaciones** (`filter`, `map`, etc.).
3. **Confirma que aún no se ha ejecutado nada** (lazy evaluation).
4. **Ejecuta una acción** como `sum()`, `mean()` o `stdev()`.
5. **Visualiza cómo Spark organiza el Job**  
   - Explicación conceptual  
   - Spark UI (si está disponible)
6. **Dibuja el DAG del Job** con los pasos aplicados.


In [5]:

#Importar Spark y crear SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("JobSparkEjemplo") \
    .getOrCreate()

sc = spark.sparkContext

#Crear un RDD numérico desde una lista
numeros = [1,2,3,4,5,6,7,8,9,10]
rdd_numeros = sc.parallelize(numeros)
print("RDD creado (lazy evaluation):", rdd_numeros)

#Aplicar transformaciones (filter y map)
rdd_filtrado = rdd_numeros.filter(lambda x: x % 2 == 0)   # Solo números pares
rdd_multiplicado = rdd_filtrado.map(lambda x: x * 2)     # Multiplicamos por 2
print("RDD con transformaciones (lazy):", rdd_multiplicado)

#Confirmar lazy evaluation
# Hasta aquí, nada se ejecuta realmente. Solo se crea el plan de ejecución (DAG interno)

#Ejecutar acciones
suma = rdd_multiplicado.sum()          # Suma
media = rdd_multiplicado.mean()        # Media
desviacion = rdd_multiplicado.stdev()  # Desviación estándar

print("Resultado de la acción sum():", suma)
print("Resultado de la acción mean():", media)
print("Resultado de la acción stdev():", desviacion)



RDD creado (lazy evaluation): ParallelCollectionRDD[10] at readRDDFromFile at PythonRDD.scala:289
RDD con transformaciones (lazy): PythonRDD[11] at RDD at PythonRDD.scala:53
Resultado de la acción sum(): 60
Resultado de la acción mean(): 12.0
Resultado de la acción stdev(): 5.656854249492381


1- DAG (Directed Acyclic Graph)

Spark construye un grafo de dependencias con todas las transformaciones (filter, map).

Nada se ejecuta hasta que se llama una acción (sum(), mean(), stdev()).

2- Stages

El DAG se divide en Stages, cada uno agrupando transformaciones que se pueden ejecutar sin mezclar datos.

Ejemplo: Stage 1 → filter + map, Stage 2 → sum/mean/stdev.

3- Tasks

Cada Stage se divide en Tasks, que se ejecutan en paralelo por partición de datos.

4- Lazy Evaluation

Las transformaciones son perezosas; solo se ejecutan al llamar una acción.

## DAG conceptual del Job Spark

```text
          DAG del Job Spark
-------------------------------------------------
[RDD original: 1-10]       <-- Datos iniciales
           |
       Stage 1: Transformaciones
   -----------------------------
   |                           |
 filter (pares)            map (*2)
   |                           |
 Task 1                     Task 2   <-- cada partición se procesa en paralelo
   |
   v
Stage 2: Acción (reduce)
   -----------------------------
   |           |             |
 sum()       mean()       stdev()   <-- acción dispara ejecución
   |           |             |
 Task 1     Task 1        Task 1    <-- combina resultados parciales
